In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import torch
from torch.utils.data import DataLoader, TensorDataset

def make_dataloader(n, batch_size):
    X = np.random.random((n, 2)) * 2 - 1
    Y = (X[:, 0] > X[:, 1] + 0.25).astype(int) + 1

    X = torch.tensor(X, dtype=torch.float)
    Y = torch.tensor(Y, dtype=torch.long)

    div1 = int(n*0.8)
    div2 = int(n*0.9)
    Xs = [X[:div1], X[div1:div2], X[div2:]]
    Ys = [Y[:div1], Y[div1:div2], Y[div2:]]

    dataset = TensorDataset(Xs[0], Ys[0])
    data_loader = DataLoader(dataset, batch_size=batch_size)
    return data_loader

In [21]:
import torch.nn as nn
from metal.mmtl.utils.metrics import acc_f1, pearson_spearman
from metal.mmtl.task import Task
from metal.mmtl.scorer import Scorer

BATCHSIZE = 32

foo_input = nn.Linear(2, 10)
bar_input = foo_input #nn.Linear(100, 7)

foo_head = nn.Linear(10, 2)
bar_head = nn.Linear(10, 2)

foo_data = make_dataloader(5000, batch_size=BATCHSIZE)
bar_data = make_dataloader(2000, batch_size=BATCHSIZE)


# custom_metrics = {
#     pearson_spearman: ["pearson_corr", "spearman_corr", "pearson_spearman"]
# }
# scorer = Scorer(["accuracy"], custom_metric_funcs=custom_metrics)

foo = Task("foo_task", {"train": foo_data, "valid": foo_data, "test": foo_data}, foo_input, foo_head)
bar = Task("bar_task", {"train": bar_data, "valid": bar_data, "test": bar_data}, bar_input, bar_head)
# baz = Task("baz_task", "baz_head", [make_dataloader(100), None, None])
tasks = [foo, bar]

In [24]:
from metal.end_model import EndModel
from metal.mmtl.metal_model import MetalModel
from metal.mmtl.trainer import MultitaskTrainer

model = MetalModel(tasks, device=-1, verbose=False)
trainer = MultitaskTrainer()
trainer.train_model(
    model, 
    tasks, 
    n_epochs=3, 
    lr=0.1, 
    progress_bar=True,
    log_unit="epochs",
    log_every=1,
    score_every=1,
#     optimizer="sgd",
    lr_scheduler="exponential",
    warmup_steps = 50,
    min_lr = 0.0001,
#     patience=10,
#     task_metrics=["foo_task/valid/acc_f1"],
    trainer_metrics=["lr"],
    test_split="test",
    grad_clip=0.01,
    writer=None,
    checkpoint=False,
    checkpoint_best=True,
    checkpoint_metric="foo_task/valid/accuracy",
    checkpoint_metric_mode="max",
)

Beginning train loop.
Expecting a total of approximately 5600 examples and 175 batches per epoch from 2 tasks.


[1.0 epo]: TRAIN:[loss=0.028] VALID:[foo_task/accuracy=0.991, bar_task/accuracy=0.991, model/lr=0.000]


[2.0 epo]: TRAIN:[loss=0.017] VALID:[foo_task/accuracy=0.992, bar_task/accuracy=0.994, model/lr=0.000]


[3.0 epo]: TRAIN:[loss=0.014] VALID:[foo_task/accuracy=0.994, bar_task/accuracy=0.995, model/lr=0.000]
Finished Training
{'bar_task/test/accuracy': 0.995,
 'foo_task/test/accuracy': 0.9935,
 'model/test/lr': 0.0001}


In [ ]:
trainer.optimizer.param_groups[0]["lr"]

In [ ]:
# for batch in foo.data_loaders["train"]:
#     X, Y = batch
#     print(model(X, ['foo_task']))
#     print(model.calculate_loss(X, Y, ['foo_task']))    
#     print(model.calculate_output(X, ['foo_task']))    
#     break

In [ ]:
model.predict_probs(foo, "train")

In [ ]:
model.predict(foo, "train")

In [ ]:
model.score(foo, "valid", "accuracy")

In [ ]:
a = {"foo": 1, "bar": 2}
list(a.keys())

In [5]:
# import os
# filepath = os.path.join(os.environ["METALHOME"], "my_model.pkl")
# model.save(filepath)

In [18]:
a = []
a[0]

IndexError: list index out of range